# Libraries Loading

In [41]:
import pandas as pd
import arabic
import pyarabic
import helpers
#from helpers import save_h5
from helpers import Clean_data
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
#########################
from numpy import array, argmax
import numpy as np
import re
import os
import re
import sys
from pyarabic.araby import strip_tatweel
from pyarabic.araby import strip_tashkeel
import pickle
##########################

input_data_path = "../data/Almoso3a_Alshe3rya/data/All_ksaied.csv"

In [31]:
import arabic
from itertools import product 
from pyarabic.araby import strip_tashkeel, strip_tatweel
import numpy as np
from numpy import array
import re
import h5py

counter = 0



def save_h5(nameOfFile,nameOfDataset,dataVar):
    h5f = h5py.File(nameOfFile, 'w')
    h5f.create_dataset(nameOfDataset, data=dataVar)
    h5f.close()

/home/mostafaalaa/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Data Loading

In [2]:
input_data = pd.read_csv(input_data_path, index_col=0)
input_data.info(memory_usage='deep')
bahr = 'Bahr'
bayt = 'Bayt_Text'
#input_data  = input_data[[bayt, bahr]]
bahr = 'البحر'
bayt = 'البيت'
input_data  = input_data[[bayt, bahr]]
input_data.columns = ['Bayt_Text','Bahr']
our_alphabets = "".join(arabic.alphabet) + "".join(arabic.tashkeel)+" "
our_alphabets = "".join(our_alphabets)
input_data['Bayt_Text'] = input_data['Bayt_Text'].apply(lambda x: re.sub(r'[^'+our_alphabets+']','',str(x))).apply(lambda x: re.sub(r'  *'," ",x)).apply(lambda x: re.sub(r'ّ+', 'ّ', x)).apply(lambda x: x.strip())
input_data.shape

/home/mostafaalaa/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1831770 entries, 0 to 1835252
Data columns (total 8 columns):
العصر           object
الشاعر          object
الديوان         object
القافية         object
البحر           object
الشطر الايسر    object
الشطر الايمن    object
البيت           object
dtypes: object(8)
memory usage: 1.8 GB


(1831770, 2)

# Data Analysis

In [3]:
input_data.iloc[:, 1].value_counts()


الطويل          405378
الكامل          363486
البسيط          239974
الخفيف          156049
الوافر          140560
الرجز           117572
&nbsp;           92334
الرمل            79015
المتقارب         63818
السريع           58084
موشح             30060
المنسرح          28357
المجتث           17884
المديد            7829
الهزج             7541
عامي              7074
المتدارك          5037
شعر التفعيلة      3232
الدوبيت           2713
شعر حر            2135
المواليا          1644
السلسلة            907
المقتضب            799
المضارع            288
Name: Bahr, dtype: int64

In [4]:
input_data.head()

,Bayt_Text,Bahr
0,خَليلَيَّ لا تَستَعجِلا أَن تَزَوَّدا وَأَن تَ...,الطويل
1,فَما لَبَثٌ يَوماً بِسابِقٍ مَغنَمٍ وَلا سُرعَ...,الطويل
2,وَإِن تُنظِراني اليَومَ أَقضِ لُبانَةً وَتَستَ...,الطويل
3,لَعَمرُكَ ما نَفسٌ بِجِدٍ رَشيدَةٍ تُؤامِرُني ...,الطويل
4,وَإِن ظَهَرَت مِنهُ قَوارِصُ جَمَّةٌ وَأَفرَعَ...,الطويل


In [7]:


elminated_classic_bohor = ['الوافر', 'المنسرح', 'المديد', 'المجتث', 'المتقارب', 'الكامل', 'الطويل', 'السريع', 'الرمل',
                         'الرجز', 'الخفيف', 'البسيط']

elminated_classic_bohor

['الوافر',
 'المنسرح',
 'المديد',
 'المجتث',
 'المتقارب',
 'الكامل',
 'الطويل',
 'السريع',
 'الرمل',
 'الرجز',
 'الخفيف',
 'البسيط']

In [8]:
full_bohor_classes = ['الوافر', 'المنسرح', 'المديد', 'المجتث', 'المتقارب', 'الكامل', 'الطويل', 'السريع', 'الرمل',
                         'الرجز', 'الخفيف', 'البسيط', 'المقتضب', 'الهزج', 'المضارع', 'المتدارك']
full_bohor_classes

['الوافر',
 'المنسرح',
 'المديد',
 'المجتث',
 'المتقارب',
 'الكامل',
 'الطويل',
 'السريع',
 'الرمل',
 'الرجز',
 'الخفيف',
 'البسيط',
 'المقتضب',
 'الهزج',
 'المضارع',
 'المتدارك']

### Data Filter

In [14]:
full_filtered_data = input_data.loc[input_data['Bahr'].isin(full_bohor_classes)]
full_filtered_data['Bayt_Text'] = full_filtered_data['Bayt_Text'].apply(pyarabic.araby.strip_tatweel)
full_filtered_data.head()

/home/mostafaalaa/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Bayt_Text,Bahr
0,خَليلَيَّ لا تَستَعجِلا أَن تَزَوَّدا وَأَن تَ...,الطويل
1,فَما لَبَثٌ يَوماً بِسابِقٍ مَغنَمٍ وَلا سُرعَ...,الطويل
2,وَإِن تُنظِراني اليَومَ أَقضِ لُبانَةً وَتَستَ...,الطويل
3,لَعَمرُكَ ما نَفسٌ بِجِدٍ رَشيدَةٍ تُؤامِرُني ...,الطويل
4,وَإِن ظَهَرَت مِنهُ قَوارِصُ جَمَّةٌ وَأَفرَعَ...,الطويل


In [13]:
eliminated_filtered_data = input_data.loc[input_data['Bahr'].isin(elminated_classic_bohor)]
eliminated_filtered_data['Bayt_Text'] = eliminated_filtered_data['Bayt_Text'].apply(pyarabic.araby.strip_tatweel)
eliminated_filtered_data.head()

/home/mostafaalaa/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Bayt_Text,Bahr
0,خَليلَيَّ لا تَستَعجِلا أَن تَزَوَّدا وَأَن تَ...,الطويل
1,فَما لَبَثٌ يَوماً بِسابِقٍ مَغنَمٍ وَلا سُرعَ...,الطويل
2,وَإِن تُنظِراني اليَومَ أَقضِ لُبانَةً وَتَستَ...,الطويل
3,لَعَمرُكَ ما نَفسٌ بِجِدٍ رَشيدَةٍ تُؤامِرُني ...,الطويل
4,وَإِن ظَهَرَت مِنهُ قَوارِصُ جَمَّةٌ وَأَفرَعَ...,الطويل


### Strip Tatweel

In [25]:
filtered_data['Bayt_Text'] = filtered_data['Bayt_Text'].apply(pyarabic.araby.strip_tatweel)

/home/mostafaalaa/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [36]:
try:
    os.makedirs("../data/Encoded/8bits/WithoutTashkeel/Eliminated/")
    os.makedirs("../data/Encoded/8bits/WithoutTashkeel/Full_Data/")
    os.makedirs("../data/Encoded/8bits/WithTashkeel/Eliminated/")
    os.makedirs("../data/Encoded/8bits/WithTashkeel/Full_Data/")
except OSError as e:
    if e.errno != errno.EEXIST:
        print("Can't create file for checkpoints or for logs please check ")
        raise
print("Paths created")


Paths created


# Encode Classes

In [37]:
full_data_label_encoder = LabelEncoder()
full_data_integer_encoded = full_data_label_encoder.fit_transform(list(full_filtered_data["Bahr"]))
# binary encode
full_data_onehot_encoder = OneHotEncoder(sparse=False)
full_data_integer_encoded = full_data_integer_encoded.reshape(len(full_data_integer_encoded), 1)
full_data_bohor_encoded = full_data_onehot_encoder.fit_transform(full_data_integer_encoded)
save_h5('../data/Encoded/8bits/WithoutTashkeel/Full_Data/full_date_Y_Meters.h5', 'Y', full_data_bohor_encoded)
save_h5('../data/Encoded/8bits/WithTashkeel/Full_Data/full_date_Y_Meters.h5', 'Y', full_data_bohor_encoded)
#saving with pickle: https://stackoverflow.com/questions/11218477/how-can-i-use-pickle-to-save-a-dict
print('saving preprocessors....')
with open("../encoders_full_dat.pickle", 'wb') as pre_file:
    pickle.dump(full_data_label_encoder, pre_file, protocol=pickle.HIGHEST_PROTOCOL)
print('saved ....')

saving preprocessors....


In [39]:
#print('loading saved encoders from %s'%(encoders_file_name))
with open("../encoders_full_dat.pickle", 'rb') as f:
    encoders = pickle.load(f)
##testing 
inverted = encoders.inverse_transform([argmax(full_filtered_data["Bahr"][1,])])
inverted

/home/mostafaalaa/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


array(['البسيط'], dtype='<U8')

In [40]:
eliminated_data_label_encoder = LabelEncoder()
eliminated_data_integer_encoded = eliminated_data_label_encoder.fit_transform(list(full_filtered_data["Bahr"]))
# binary encode
eliminated_data_onehot_encoder = OneHotEncoder(sparse=False)
eliminated_data_integer_encoded = eliminated_data_integer_encoded.reshape(len(eliminated_data_integer_encoded), 1)
eliminated_data_bohor_encoded = eliminated_data_onehot_encoder.fit_transform(eliminated_data_integer_encoded)
save_h5('../data/Encoded/8bits/WithoutTashkeel/Eliminated/Eliminated_data_Y_Meters.h5', 'Y', eliminated_data_bohor_encoded)
save_h5('../data/Encoded/8bits/WithTashkeel/Eliminated/Eliminated_data_Y_Meters.h5', 'Y', eliminated_data_bohor_encoded)
#saving with pickle: https://stackoverflow.com/questions/11218477/how-can-i-use-pickle-to-save-a-dict
print('saving preprocessors....')
with open("../encoders_eliminated_data.pickle", 'wb') as pre_file:
    pickle.dump(eliminated_data_bohor_encoded, pre_file, protocol=pickle.HIGHEST_PROTOCOL)


saving preprocessors....


## Load Omar Code for data cleansing in case Tashkeel

In [18]:
import pandas as pd
import re
import sys
import helpers
from pyarabic.araby import strip_tatweel
sys.path.append('.') # path of arabic file
import arabic 
vectoriz_function = helpers.string_with_tashkeel_vectorizer
counter = 0

def separate_token_with_dicrites(token):
    """gets a token(string) with taskeel, and returns a list of strings,
    each string in the list represents each character in the token with its own
    tashkeel.
    Args:
        token (str): string represents a word or aya or sura
    Returns:
        [str]: a list contains the token characters with their tashkeel.
    """
    token_without_tatweel = strip_tatweel(token)
    hroof_with_tashkeel = []
    for index,i in enumerate(token):
        if(((token[index] in (arabic.alphabet or arabic.alefat or arabic.hamzat
)) or token[index] is ' ' or  token[index] is "\n" )):
            k = index
            harf_with_taskeel = token[index]
            while((k+1) != len(token) and (token[k+1] in (arabic.tashkeel or 
            arabic.harakat or arabic.shortharakat or arabic.tanwin))):
                harf_with_taskeel =harf_with_taskeel+""+token[k+1]
                k = k + 1
            index = k
            hroof_with_tashkeel.append(harf_with_taskeel)
    return hroof_with_tashkeel


def apply_cleaning(s):
    global counter
    try:
        global vectoriz_function
        vectoriz_function(s)
        #print(counter)
        counter+=1
        return s
    except:
        s = solve_conflect(s)
        #print(counter)
        counter+=1
        return s
        #helpers.string_with_tashkeel_vectorizer(s)
        

def clean_fun(s):
    if " " in s:
        return " "
    
    non_remove = arabic.fatha+"|"+arabic.damma+"|"+arabic.kasra+"|"+arabic.sukun
    remove = arabic.dammatan+"|"+arabic.fathatan+"|"+arabic.kasratan
    tashkiel  = re.compile(r'('"("+non_remove+")"+arabic.shadda+")")
    tanwine  =  re.compile(r'('"("+remove+")"+arabic.shadda+")")
    tanwine_  =  re.compile(r'('+arabic.shadda+"("+remove+")"+")")
    spaces_w_tshkieel = re.compile(r'( ('+"|".join(arabic.tashkeel)+'))')
    tanwine_2 = re.compile(r'(('+non_remove+')('+remove+')'')')
    tow_tashkeel = re.compile(r'(('+non_remove+')('+non_remove+')'')')
    tow_tashkeel_ = re.compile(r'(('+remove+')('+remove+')'')')
    
    lis=list(s)
    for m in tashkiel.finditer(s):
        lis[m.start()] , lis[m.start()+1] = lis[m.start()+1] , lis[m.start()]
    for m in tanwine_.finditer(s):
        del lis[m.start()]
    for m in tanwine.finditer(s):
        del lis[m.start()] 
    for m in tow_tashkeel.finditer(s):
        del lis[m.start()]
    for m in tow_tashkeel_.finditer(s):
        del lis[m.start()]
    for m in spaces_w_tshkieel.finditer(s):
        del lis[m.start()+1]
    for m in tanwine_2.finditer(s):
        del lis[m.start()+1]
    return "".join(lis)


def solve_conflect(s):
    return "".join([clean_fun(c) for c in separate_token_with_dicrites(s)])


def Clean_data(data_frame,verse_column_name='Bayt_Text'):
    our_alphabets = "".join(arabic.alphabet) + "".join(arabic.tashkeel)+" "
    our_alphabets = "".join(our_alphabets)
    data_frame[verse_column_name]    = data_frame[verse_column_name] .apply(lambda x: re.sub(r'[^'+our_alphabets+']','',str(x))).apply(lambda x: re.sub(r'  *'," ",x)).apply(lambda x: re.sub(r'ّ+', 'ّ', x)).apply(lambda x: x.strip())
    data_frame[verse_column_name] = data_frame[verse_column_name].apply(apply_cleaning)
    return data_frame



### Clean the Data

In [19]:
full_cleaned_data_with_tashkeel = Clean_data(filtered_data,'Bayt_Text')
data_with_Tashkeel_cleaned = cleaned_data_with_tashkeel['Bayt_Text']


data_with_Tashkeel_cleaned.shape
#data_with_Tashkeel_cleaned.columns

/home/mostafaalaa/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/mostafaalaa/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(1691671,)

In [ ]:
maximum_bayt_len = np.max((data_with_Tashkeel_cleaned.apply(separate_token_with_dicrites).apply(len)))
maximum_bayt_len

84

# Encode Bayt Text

In [ ]:
#encoded_byot = data_with_Tashkeel_cleaned.apply(lambda x: helpers.string_with_tashkeel_vectorizer(x, maximum_bayt_len))
encoded_byot = data_with_Tashkeel_cleaned.apply(lambda x: helpers.string_with_tashkeel_vectorizer_OneHot(x, maximum_bayt_len))
len(encoded_byot[1])

In [ ]:
len(encoded_byot[1])

In [ ]:
# Reshaping the heck!!
reshaped_data_matrix_with_tashkeel = np.stack(encoded_byot,axis=0)

## Save Encoding Bayt Text

In [ ]:
save('full_data_matrix_with_tashkeel_onehotEncoding.h5', 'X', reshaped_data_matrix_with_tashkeel) 